### MNIST(80) revised augmentation -> CIFAR (5/epocaMNIST)
size = 224
xresnet34deep, pretrained=true
fit
5% do dataset
fp16

In [1]:
from fastai.vision.all import *
from datetime import datetime
import torch
torch.cuda.is_available()

True

In [2]:
path = Path('/home/fredguth/.fastai/data/cifar10_mnist')

In [3]:
def label_func(f):
    return (str(f).split("_y")[1:][0][0])

def noise_func(f):
    return (str(f).split("_n")[1:][0][0])

In [4]:
def get_dls(task="CIFAR", size=64):
    dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                   splitter  = RandomSubsetSplitter(train_sz=0.05, valid_sz=0.05, seed=42),
                   get_items = get_image_files,
                   get_y = label_func if (task == 'CIFAR') else noise_func,
                   batch_tfms= [*aug_transforms(size=size, min_scale=0.75, do_flip=False, pad_mode="zeros"), Normalize],
                   n_inp     = 1 )
    return dblock.dataloaders(path, num_workers=4)

In [5]:
class HeadBodyNetwork(Module):
    def __init__(self, body, head):
        self.head = head
        self. body = body
    def forward(self, x):
        return self.head(self.body(x))

In [6]:
def headbody_splitter(m): return L(m.head, m.body).map(params)

In [7]:
class RunMNIST(Callback):
    def after_epoch(self):  
        epoch = self.epoch
        backbone = copy(self.learn.model[0])
        head =  create_head(num_features_model(backbone),10)
        net = HeadBodyNetwork(head=head, body=backbone)
        l =Learner(
                           dls=get_dls(task="CIFAR", size=32), 
                           model=net,
                           splitter=headbody_splitter,
                           loss_func=F.cross_entropy, 
                           metrics=accuracy, 
                           cbs=[CSVLogger(fname=f"results/ex7/cifar_after_mnist_e{epoch}.csv", append=True)])
        l.freeze()
        l.fit(3,0.002)

In [8]:
m = Learner(dls=get_dls(task="MNIST", size=224), model=resnet18)

In [9]:
n = create_body(m.model)

In [10]:
body = nn.Sequential(n, create_head(512, 64))
head = create_head(64,10)

In [11]:
net = HeadBodyNetwork(body, head)

In [12]:
mnist_learner= Learner(
                           dls=get_dls(task="CIFAR", size=32), 
                           model=net,
                           splitter=headbody_splitter,
                           loss_func=F.cross_entropy, 
                           metrics=accuracy, 
                           cbs=[CSVLogger(fname=f"results/ex7/mnist.csv", append=True), RunMNIST]
                          )

In [13]:
mnist_learner.to_fp16()

In [14]:
mnist_learner.fit(40,0.001)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: non-empty 3D or 4D (batch mode) tensor expected for input

In [ ]:
import pandas as pd
import glob

In [ ]:
!pwd

In [ ]:
all_files = glob.glob("/home/fredguth/unb/cifar_mnist/results/ex6/cifar*.csv")

In [ ]:
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

In [ ]:
frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
cifar_results = frame.loc[frame['epoch'] == 2]

In [ ]:
cifar_results = cifar_results.reset_index().drop(columns=['index', 'epoch'])

In [ ]:
df = pd.read_csv("/home/fredguth/unb/cifar_mnist/results/ex6/mnist.csv", index_col=None, header=0)

In [ ]:
result = pd.concat([cifar_results['valid_loss'], df['valid_loss']], axis=1, keys=['CIFAR', 'MNIST'])

In [ ]:
df.to_excel("/home/fredguth/unb/cifar_mnist/results/ex6/mnist.xlsx")

In [ ]:
cifar_results.to_excel("results/ex6/ex6_cifar_results.xlsx")

In [ ]:
result.to_excel("/home/fredguth/unb/cifar_mnist/results/ex6/ex6.xlsx")

In [ ]:
plt.scatter(x=result['CIFAR'].tolist(), y=result['MNIST'].tolist())
plt.plot(result['CIFAR'].tolist(), result['MNIST'].tolist(), '-o')
plt.show()

In [ ]:
a, b= (result['CIFAR'].tolist(), result['MNIST'].tolist())

In [ ]:
from matplotlib.collections import LineCollection

In [ ]:
x = np.array(a)
y = np.array(b)
x  = np.array([ (i - x.min())/x.max() for i in x])
y  = np.array([ (i - y.min())/y.max() for i in y])
x  = np.array([ x.max()-i  for i in x])
y  = np.array([ y.max()-i  for i in y])

In [ ]:
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
lc = LineCollection(segments, colors=linecolors)
fig1 = plt.figure()
plt.gca().add_collection(lc)
plt.xlim(x.min(), x.max())
plt.ylim(y.min(), y.max())

In [ ]:

result = pd.concat([cifar_results['accuracy'], df['accuracy']], axis=1, keys=['CIFAR', 'MNIST'])

In [ ]:
a, b= (result['CIFAR'].tolist(), result['MNIST'].tolist())

In [ ]:
x = np.array(a)
y = np.array(b)
x  = np.array([ (i - x.min())/x.max() for i in x])
y  = np.array([ (i - y.min())/y.max() for i in y])
# x  = np.array([ x.max()-i  for i in x])
# y  = np.array([ y.max()-i  for i in y])

In [ ]:
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
lc = LineCollection(segments, colors=linecolors)
fig1 = plt.figure()
plt.gca().add_collection(lc)
plt.xlim(x.min(), x.max())
plt.ylim(y.min(), y.max())